In [1]:
import pandas as pd

In [2]:
data_url = "https://github.com/killerWhale0917/Recsys-PyTorch-Implementation/raw/main/ratings.csv"
ml_data = pd.read_csv(data_url)

ml_data.rename(columns={'userId': 'user_id', 'movieId': 'movie_id'}, inplace=True)
ml_data['interaction'] = (ml_data['rating'] >= 3.0).astype(int)
ml_data.drop(columns=['timestamp', 'rating'], inplace=True)
ml_data

,user_id,movie_id,interaction
0,1,1,1
1,1,3,1
2,1,6,1
3,1,47,1
4,1,50,1
...,...,...,...
100831,610,166534,1
100832,610,168248,1
100833,610,168250,1
100834,610,168252,1


In [3]:
original_movie_id = ml_data['movie_id'].unique().tolist()
new_movie_id = list(range(1, len(original_movie_id) + 1))
movie_id_mapping = dict(zip(original_movie_id, new_movie_id))

ml_data['movie_id'] = ml_data['movie_id'].map(movie_id_mapping)

In [4]:
ml_data

,user_id,movie_id,interaction
0,1,1,1
1,1,2,1
2,1,3,1
3,1,4,1
4,1,5,1
...,...,...,...
100831,610,3121,1
100832,610,2036,1
100833,610,3122,1
100834,610,1393,1


In [5]:
import numpy as np

all_df = pd.DataFrame({ "user_id": np.repeat(ml_data['user_id'].unique(), ml_data['movie_id'].nunique()),
                        "movie_id": np.tile(ml_data["movie_id"].unique(), ml_data['user_id'].nunique())})

merged_df = pd.merge(all_df, ml_data, on = ['user_id', 'movie_id'], how='left')
# merged_df
merged_df['interaction'].fillna(0, inplace=True)
merged_df

,user_id,movie_id,interaction
0,1,1,1.0
1,1,2,1.0
2,1,3,1.0
3,1,4,1.0
4,1,5,1.0
...,...,...,...
5931635,610,9720,0.0
5931636,610,9721,1.0
5931637,610,9722,1.0
5931638,610,9723,1.0


In [6]:
import torch
from torch.utils.data import TensorDataset

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

users = torch.tensor(merged_df['user_id'].values)
items = torch.tensor(merged_df['movie_id'].values)
feedback = torch.tensor(merged_df['interaction'].values)

users = users.to(device)
items = items.to(device)
feedback = feedback.to(device)

dataset = TensorDataset(users, items, feedback)

cuda:0


In [7]:
dataset

In [8]:
import torch
import torch.nn as nn

class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, latent_dim, hidden_dim):
        super(NeuMF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, latent_dim)
        self.item_embedding = nn.Embedding(num_items, latent_dim)
        self.mlp = nn.Sequential(
            nn.Linear(latent_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, user_idx, item_idx):
        user_latent = self.user_embedding(user_idx)
        item_latent = self.item_embedding(item_idx)
        latent = torch.cat((user_latent, item_latent), dim=-1)
        prediction = self.mlp(latent)
        return prediction

In [9]:
num_users = ml_data['user_id'].nunique()
num_items = ml_data['movie_id'].nunique()

print (f"# of users : {num_users}\n# of items : {num_items}")

# of users : 610
# of items : 9724


In [10]:
model = NeuMF(num_users, num_items, latent_dim=100, hidden_dim=50)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [11]:
from tqdm import tqdm
model.to(device)

for user_idx, item_idx, interaction in tqdm(dataset):
    prediction = model(user_idx, item_idx)
    loss = criterion(prediction.float(), interaction.float())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  0%|          | 0/5931640 [00:00<?, ?it/s]c:\Users\moon\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|          | 9723/5931640 [00:54<9:12:24, 178.67it/s] 


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`